# Введение в визуализацию данных с помощью Altair

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Введение%20в%20визуализацию%20данных%20с%20помощью%20Altair.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/pyviz.jpg" height="600px" width="800px" >

[*Источник картинки*](https://pyviz.org/overviews/index.html)

Чтобы не отставать от последних трендов в области визуализации, я недавно услышал об [*Altair*](https://altair-viz.github.io/), который называет себя *"библиотекой декларативной статистической визуализации для Python"*. 

> Оригинал статьи Криса [тут](https://pbpython.com/altair-intro.html)

Меня особенно заинтересовало то, что он разработан [Брайаном Грейнджером](https://twitter.com/ellisonbg) (*Brian Granger*) и [Джейком Вандерпласом](https://twitter.com/jakevdp) (*Jake Vanderplas*). Брайан является основным разработчиком проекта *IPython* и очень активен в научном сообществе *Python*. Джейк также активен в научном сообществе питонистов и написал прекрасную книгу ["Python Data Science Handbook"](https://jakevdp.github.io/PythonDataScienceHandbook/). Оба эти человека чрезвычайно опытны и хорошо осведомлены о *Python* и различных инструментах в его научной экосистеме. Из-за их прошлого мне было очень любопытно посмотреть, как они подошли к этой проблеме.

## Общее описание

Одна из уникальных концепций дизайна *Altair* заключается в том, что он использует спецификацию [Vega-Lite](https://vega.github.io/vega-lite/) для создания "красивых и эффективных визуализаций с минимальным количеством кода". 

> Vega-Lite - это [грамматика высокого уровня интерактивной графики](https://vega.github.io/vega-lite/tutorials/getting_started.html). Она предоставляет краткий декларативный синтаксис JSON для создания выразительного набора визуализаций для анализа и представления данных.

Что это значит? 

*Altair* предоставляет *Python API* для декларативного построения статистических визуализаций. 

Под статистической визуализацией понимается:

- Источником данных является `DataFrame`, который состоит из столбцов с разными типами данных (количественные, порядковые, номинальные и дата/время).
- `DataFrame` имеет *аккуратный* [tidy](http://vita.had.co.nz/papers/tidy-data.pdf) формат, где строки соответствуют выборкам, а столбцы соответствуют наблюдаемым переменным.
- Данные сопоставляются с визуальными свойствами (положение, цвет, размер, форма и т. д.) с помощью операции группировки Pandas и SQL.
- API Altair не содержит фактического кода визуализации, но вместо этого генерирует JSON структуры данных в соответствии со спецификацией *Vega-Lite*. Для удобства *Altair* может дополнительно использовать [ipyvega](https://github.com/vega/ipyvega) для плавного отображения клиентских рендеров в Jupyter блокноте.

*Altair* пытается интерпретировать переданные ему данные и проделать некоторые разумные предположения о том, как их отображать. Делая разумные предположения, пользователь может тратить больше времени на изучение данных, чем на попытки разработать сложный API для их отображения.

Прежде чем двигаться дальше, я хотел бы выделить еще один уникальный аспект *Altair*, связанный с ожидаемым форматом данных. Как описано выше, *Altair* ожидает, что все данные будут в *аккуратном (tidy) формате*. 

Общая идея заключается в том, что вы преобразуете свои данные в соответствующий формат, а затем используете API *Altair* для выполнения различных группировок или других методов сводки данных для вашей конкретной ситуации. Новым пользователям может потребоваться некоторое время, чтобы привыкнуть к этому. Тем не менее, я думаю, что в долгосрочной перспективе это хороший навык, и вложения в обработку данных (при необходимости) окупятся, в конце концов, путем обеспечения согласованного процесса визуализации данных. 

# Обзор возможностей Altair

> Оригинал документации [тут](https://github.com/altair-viz/altair-tutorial)

Установим необходимые модули:

In [ ]:
#!pip3 install altair

In [ ]:
#!pip3 install vega_datasets

Начнем с демонстрации возможностей *Altair*. 

В этом разделе поверхностно рассматриваются многие концепции, например, `data`, `marks`, `encodings`, `aggregation`, `data types`, `selections` и т. д. Позже мы вернемся к более глубокому рассмотрению каждой из них, поэтому не беспокойтесь, если покажется, что все идет слишком быстро!

> *Altair* строится на [спецификации Vega-Lite](https://vega.github.io/vega-lite/tutorials/getting_started.html) и вся терминология взята оттуда.

## Изучение набора данных автомобилей

Начнем с импорта пакета *Altair*:

In [ ]:
import altair as alt

Теперь воспользуемся пакетом [vega_datasets](https://github.com/altair-viz/vega_datasets), чтобы загрузить набор данных:

In [ ]:
from vega_datasets import data

cars = data.cars()
cars.head()

Используя *Altair*, можем исследовать эти данные.

Самая простая [диаграмма](https://altair-viz.github.io/user_guide/generated/toplevel/altair.Chart.html#altair.Chart) (*chart*) содержит набор данных вместе с меткой (*mark*) для представления каждой строки:

In [ ]:
alt.Chart(cars).mark_point()

Это довольно глупая диаграмма, потому что она состоит из `406` точек, расположенных друг над другом.

Чтобы сделать ее более интересной, необходимо *закодировать* (`encode`) столбцы данных в визуальные элементы графика (*plot*), например, положение `x`, положение `y`, `size`, `color` и т. д.

Давайте закодируем *мили на галлон* (*miles per gallon*) по оси `x` с помощью метода [`encode()`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.Chart.html#altair.Chart.encode):

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon'
)

Немного лучше, но `point` (*точечная*) маркировка, вероятно, не самая лучшая для такой одномерной диаграммы.

Вместо этого попробуем задать `tick` маркировку:

In [ ]:
alt.Chart(cars).mark_tick().encode(
    x='Miles_per_Gallon'
)

Можем развернуть в 2D-диаграмму, также закодировав значение `y`. 

Вернемся к использованию `point` (*точечной*) маркировки и поместим `Horsepower` (*мощность в лошадиных силах*) по оси `y`:

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower'
)

Одна из самых приятных особенностей *Altair* - это грамматика взаимодействия, которую он предоставляет. 

Самый простой вид взаимодействия - это возможность панорамировать (*pan*) и масштабировать (*zoom*) диаграммы; их можно включить с помощью метода `interactive()`:

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower'
).interactive()

Это позволяет нажимать и перетаскивать, а также использовать прокрутку/масштабирование для увеличения и уменьшения масштаба диаграммы.

Позже мы увидим и другие варианты взаимодействия.

Двухмерный график (*2D plot*) позволяет кодировать два измерения данных. 

Давайте посмотрим, как использовать *цвет* (*color*) для кодирования третьего измерения (`Origin`):

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Origin'
)

Обратите внимание, что когда мы используем категориальное значение (*categorical value*) для цвета, Altair выбирает соответствующую цветовую карту для категориальных данных.

Посмотрим, что происходит, когда мы используем непрерывное значение цвета (`Acceleration`):

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Acceleration'
)

Непрерывный цвет формирует цветовую шкалу, подходящую для непрерывных данных.

А как насчет промежуточного случая: упорядоченные категории, например количество цилиндров (`Cylinders`)?

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Cylinders'
)

*Altair* по-прежнему выбирает непрерывное значение, потому что количество цилиндров числовое.

Можем улучшить это, указав, что данные следует рассматривать как дискретное упорядоченное значение, добавив `":O"` (`"O"` для "порядковых" или "упорядоченных категорий") после кодирования (*encoding*):

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Cylinders:O'
)

Теперь у нас есть дискретная надпись (*legend*) с упорядоченным цветовым отображением.

Давайте быстро вернемся к нашей одномерной диаграмме (*1D chart*) *миль на галлон*:

In [ ]:
alt.Chart(cars).mark_tick().encode(
    x='Miles_per_Gallon',
)

Другой способ представления этих данных - создание *гистограммы*: объединить (*to bin*) данные `x` и отобразить счетчик (*count*) по оси `y`. 

Во многих библиотеках это делается с помощью специального метода `hist()`. В *Altair* такое объединение (*binning*) и агрегация является частью декларативного API.

Чтобы выйти за рамки простого имени поля, мы используем `alt.X()` для кодирования `x`, и `count()` для кодирования `y`:

In [ ]:
alt.Chart(cars).mark_bar().encode(
    x=alt.X('Miles_per_Gallon', bin=True),
    y='count()'
)

Если нам нужен больший контроль над ячейками (bins), мы можем использовать `alt.Bin` для настройки параметров ячейки:

In [ ]:
alt.Chart(cars).mark_bar().encode(
    x=alt.X('Miles_per_Gallon', bin=alt.Bin(maxbins=30)),
    y='count()'
)

Если мы применим другое кодирование (например, `color`), данные будут автоматически сгруппированы в каждой ячейке:

In [ ]:
alt.Chart(cars).mark_bar().encode(
    x=alt.X('Miles_per_Gallon', bin=alt.Bin(maxbins=30)),
    y='count()',
    color='Origin'
)

Если вы предпочитаете отдельный график для каждой категории, то может помочь кодирование `column`:

In [ ]:
alt.Chart(cars).mark_bar().encode(
    x=alt.X('Miles_per_Gallon', bin=alt.Bin(maxbins=30)),
    y='count()',
    color='Origin',
    column='Origin'
)

Биннинг и агрегация также работают в двух измерениях; мы можем использовать `rect` маркер и визуализировать количество (*count*) с помощью цвета (*color*):

In [ ]:
alt.Chart(cars).mark_rect().encode(
    x=alt.X('Miles_per_Gallon', bin=True),
    y=alt.Y('Horsepower', bin=True),
    color='count()'
)

Агрегации могут быть не просто количеством (*counts*); мы также можем агрегировать и вычислять среднее (*mean*) значение третьего измерения в каждой ячейке:

In [ ]:
alt.Chart(cars).mark_rect().encode(
    x=alt.X('Miles_per_Gallon', bin=True),
    y=alt.Y('Horsepower', bin=True),
    color='mean(Weight_in_lbs)'
)

До сих пор мы игнорировали столбец `date`, но интересно увидеть временной тренд, например, *миль на галлон*:

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Year',
    y='Miles_per_Gallon'
)

Ежегодное есть несколько автомобилей, и данные во многом совпадают. 

Можем немного очистить их, построив среднее для каждого значения `x`:

In [ ]:
alt.Chart(cars).mark_line().encode(
    x='Year',
    y='mean(Miles_per_Gallon)',
)

В качестве альтернативы можем изменить метку на `area` (*площадь*) и использовать метки `ci0` и `ci1` для построения доверительного интервала оценки среднего:

In [ ]:
alt.Chart(cars).mark_area().encode(
    x='Year',
    y='ci0(Miles_per_Gallon)',
    y2='ci1(Miles_per_Gallon)'
)

Давайте немного скорректируем эту диаграмму: добавим непрозрачности (*opacity*), цвета по стране происхождения (`Origin`), увеличим ширину и добавим более понятный заголовок оси:

In [ ]:
alt.Chart(cars).mark_area(opacity=0.3).encode(
    x=alt.X('Year', timeUnit='year'),
    y=alt.Y('ci0(Miles_per_Gallon)', axis=alt.Axis(title='Miles per Gallon')),
    y2='ci1(Miles_per_Gallon)',
    color='Origin'
).properties(
    width=800
)

Наконец, мы можем использовать API слоев *Altair* для наложения линейной диаграммы, представляющей среднее значение, поверх диаграммы с областями, представляющей доверительный интервал:

In [ ]:
spread = alt.Chart(cars).mark_area(opacity=0.3).encode(
    x=alt.X('Year', timeUnit='year'),
    y=alt.Y('ci0(Miles_per_Gallon)', axis=alt.Axis(title='Miles per Gallon')),
    y2='ci1(Miles_per_Gallon)',
    color='Origin'
).properties(
    width=800
)

lines = alt.Chart(cars).mark_line().encode(
    x=alt.X('Year', timeUnit='year'),
    y='mean(Miles_per_Gallon)',
    color='Origin'
).properties(
    width=800
)

spread + lines

Вернемся к нашему графику рассеяния и посмотрим на другие типы интерактивности, которые предлагает *Altair*:

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Origin'
)

Напомним, что вы можете добавить `interactive()` в конец диаграммы, чтобы включить самые простые интерактивные шкалы:

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Origin'
).interactive()

*Altair* предоставляет обобщенный `selection` API для создания интерактивных графиков; например, далее мы создаем выбор интервала (*interval selection*):

In [ ]:
interval = alt.selection_interval()

alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Origin'
).add_selection(
    interval
)

Сейчас этот выбор ничего не делает, но мы можем изменить это, задав цвет для выбора:

In [ ]:
interval = alt.selection_interval()

alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color=alt.condition(interval, 'Origin', alt.value('lightgray'))
).add_selection(
    interval
)

Хорошая особенность `selection` API заключается в том, что он *автоматически* применяется ко всем составным диаграммам; например, далее мы можем объединить две диаграммы по горизонтали, и, поскольку они имеют одинаковый `selection`, то обе реагируют одинаково:

In [ ]:
interval = alt.selection_interval()

base = alt.Chart(cars).mark_point().encode(
    y='Horsepower',
    color=alt.condition(interval, 'Origin', alt.value('lightgray')),
    tooltip='Name'
).add_selection(
    interval
)

base.encode(x='Miles_per_Gallon') | base.encode(x='Acceleration')

С `selections` мы можем делать еще более сложные вещи. 

Например, давайте сделаем гистограмму количества машин по `Origin` и добавим (*stack*) ее на нашу диаграмму рассеяния:

In [ ]:
interval = alt.selection_interval()

base = alt.Chart(cars).mark_point().encode(
    y='Horsepower',
    color=alt.condition(interval, 'Origin', alt.value('lightgray')),
    tooltip='Name'
).add_selection(
    interval
)

hist = alt.Chart(cars).mark_bar().encode(
    x='count()',
    y='Origin',
    color='Origin'
).properties(
    width=800,
    height=80
).transform_filter(
    interval
)

scatter = base.encode(x='Miles_per_Gallon') | base.encode(x='Acceleration')

scatter & hist

## Простые диаграммы: основные концепции

Цель данного раздела - научить вас основным концепциям, необходимым для создания базовой диаграммы в *Altair*:

- **Данные** (*data*), **метки** (*marks*) и **кодирование** (*encodings*): три основных элемента диаграммы *Altair*.
- **Типы кодирования**: `Q` (количественное), `N` (номинальное), `O` (порядковое), `T` (временное), которые определяют визуальное представление кодирования.
- **Биннинг и агрегирование**: которые позволяют контролировать аспекты представления данных в *Altair*.

Начнем с импорта *Altair*:

In [ ]:
import altair as alt

Важнейшими элементами диаграммы *Altair* являются данные (*data*), метка (*mark*) и кодировка (*encoding*).

Формат, в котором они указаны, будет выглядеть примерно так:

```python
alt.Chart(data).mark_point().encode(
    encoding_1='column_1',
    encoding_2='column_2',
    # etc.
)
```

Давайте посмотрим на эти части.

### Данные

Данные в *Altair* построены на основе [`Dataframe`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) *pandas*. 

Далее будем использовать набор данных автомобилей, который загрузим с помощью пакета `vega_datasets`:

In [ ]:
from vega_datasets import data
cars = data.cars()

cars.head()

Ожидается, что данные в *Altair* будут в [аккуратном формате](http://vita.had.co.nz/papers/tidy-data.pdf); другими словами:

- каждая строка - это наблюдение;
- каждый столбец - это переменная.

> Дополнительную информацию см. в [документации по данным *Altair*](https://altair-viz.github.io/user_guide/data.html).

### Объект Chart

Определив данные, вы можете создать экземпляр фундаментального объекта *Altair* - `Chart`. По сути, `Chart` - это объект, который знает, как генерировать JSON словарь, представляющий данные и кодировки визуализации, которые могут быть отправлены в блокнот и обработаны JavaScript библиотекой Vega-Lite. 

Давайте посмотрим, как выглядит это JSON-представление, используя только первую строку данных:

In [ ]:
cars1 = cars.iloc[:1]
alt.Chart(cars1).mark_point().to_dict()

На этом этапе диаграмма включает представление фрейма данных в JSON формате, какой тип метки использовать, а также некоторые метаданные, которые включаются в каждый вывод диаграммы.

### Метка

Мы можем решить, какую метку мы хотели бы использовать для представления наших данных. В предыдущем примере мы можем выбрать `point` (*точечную*) метку для представления данных в виде точки на графике:

In [ ]:
alt.Chart(cars).mark_point()

В результате получается визуализация с одной точкой на строку в данных, хотя это не особенно интересно: все точки располагаются друг над другом!

Полезно еще раз изучить JSON вывод:

In [ ]:
alt.Chart(cars1).mark_point().to_dict()

Обратите внимание, что теперь помимо данных в спецификацию включена информация о типе метки.

Есть ряд доступных меток, которые вы можете использовать. 

Вот некоторые из наиболее распространенных:

* `mark_point()` 
* `mark_circle()`
* `mark_square()`
* `mark_line()`
* `mark_area()`
* `mark_bar()`
* `mark_tick()`

Вы можете получить полный список методов `mark_*`, используя функцию завершения табуляции в *Jupyter*, в любой ячейке просто введите:

    alt.Chart.mark_
    
с последующим нажатием клавиши табуляции, чтобы увидеть доступные параметры.

### Кодировки

Следующим шагом является добавление к диаграмме *каналов визуального кодирования* (или для краткости *кодирования*). Канал кодирования определяет, как данный столбец должен отображаться на визуальные свойства визуализации. 

Некоторые из наиболее часто используемых визуальных кодировок:

- `x`: значение оси x
- `y`: значение оси y
- `color`: цвет метки
- `opacity`: прозрачность/непрозрачность метки
- `shape`: форма метки
- `size`: размер метки
- `row`: строка в сетке фасетных графиков
- `column`: столбец в сетке фасетных графиков

> Полный список кодировок см. в [документации](https://altair-viz.github.io/user_guide/encoding.html).

Визуальные кодировки могут быть созданы с помощью метода `encode()` объекта `Chart`. Например, мы можем начать с сопоставления оси `y` диаграммы со столбцом `Origin`:

In [ ]:
alt.Chart(cars).mark_point().encode(
    y='Origin'
)

Результатом является одномерная визуализация, представляющая значения, принятые из `Origin`, с точками в каждой категории поверх друг друга. 

Как и выше, мы можем просмотреть JSON данные, созданные для этой визуализации:

In [ ]:
alt.Chart(cars1).mark_point().encode(
    x='Origin'
).to_dict()

Результат такой же, как и выше, с добавлением ключа `'encoding'`, который указывает канал визуализации (`y`), имя поля (`Origin`) и тип переменной (`nominal`). Мы обсудим эти типы данных чуть позже.

Визуализацию можно сделать более интересной, добавив в кодировку еще один канал: давайте закодируем `Miles_per_Gallon` как позицию `x`:

In [ ]:
alt.Chart(cars).mark_point().encode(
    y='Origin',
    x='Miles_per_Gallon'
)

Вы можете добавить столько кодировок, сколько захотите, при этом каждая кодировка будет сопоставлена столбцу данных. 

Например, далее мы раскрасим точки по `Origin` и построим график `Miles_per_gallon` против `Year`:

In [ ]:
alt.Chart(cars).mark_point().encode(
    color='Origin',
    y='Miles_per_Gallon',
    x='Year'
)

### Упражнение: изучение данных

Теперь, когда вы знаете основы (данные, кодировки, метки), потратьте немного времени и попробуйте создать несколько графиков!

В частности, я бы предложил попробовать различные комбинации из следующего:

- Метки: ``mark_point()``, ``mark_line()``, ``mark_bar()``, ``mark_text()``, ``mark_rect()``...
- Столбцы данных: ``'Acceleration'``, ``'Cylinders'``, ``'Displacement'``, ``'Horsepower'``, ``'Miles_per_Gallon'``, ``'Name'``, ``'Origin'``, ``'Weight_in_lbs'``, ``'Year'``
- Кодировки: ``x``, ``y``, ``color``, ``shape``, ``row``, ``column``, ``opacity``, ``text``, ``tooltip``...

В частности, подумайте о следующем:

- Какие кодировки подходят для непрерывных количественных значений?
- Какие кодировки подходят для дискретных, категориальных (то есть номинальных) значений?

### Типы кодирования

Одна из центральных идей *Altair* заключается в том, что библиотека выбирает подходящие значения по умолчанию для вашего типа данных.

Основные типы данных, поддерживаемые *Altair*, следующие:

<table>
  <tr>
    <th>Тип данных</th>
    <th>Код</th>
    <th>Описание</th>
  </tr>
  <tr>
    <td>quantitative</td>
    <td>Q</td>
    <td>Числовая величина (действительная)</td>
  </tr>
  <tr>
    <td>nominal</td>
    <td>N</td>
    <td>Наименование / Неупорядоченный категориальный</td>
  </tr>
  <tr>
    <td>ordinal</td>
    <td>O</td>
    <td>Упорярядоченный категориальный</td>
  </tr>
  <tr>
    <td>temporal</td>
    <td>T</td>
    <td>Дата / время</td>
  </tr>
</table>

Когда вы указываете данные в виде *фрейма данных* *pandas*, эти типы *автоматически определяются* *Altair*.

Когда вы указываете данные как URL, вы должны *вручную указать* типы данных для каждого из столбцов.

Давайте посмотрим на простой график, содержащий три столбца данных об автомобилях:

In [ ]:
alt.Chart(cars).mark_tick().encode(
    x='Miles_per_Gallon',
    y='Origin',
    color='Cylinders'
)

Вопросы:

- какой тип данных лучше всего подходит для `Miles_per_Gallon`?
- какой тип данных лучше всего подходит для `Origin`?
- какой тип данных лучше всего подходит для `Cylinders`?

Давайте добавим сокращения для каждого из этих типов данных в нашу спецификацию, используя однобуквенные коды выше (например, измените `"Miles_per_Gallon"` на `"Miles_per_Gallon: Q"`, чтобы явно указать, что это количественный тип):

In [ ]:
alt.Chart(cars).mark_tick().encode(
    x='Miles_per_Gallon:Q',
    y='Origin:N',
    color='Cylinders:O'
)

Обратите внимание, как только мы изменим тип данных для `Cylinders` на порядковый, график изменится.

При использовании *Altair* полезно выработать привычку всегда указывать эти типы явно, потому что это обязательно при работе с данными, загруженными из файла или URL.

### Упражнение: добавление явных типов

Ниже приведены несколько простых диаграмм, созданных с использованием набора данных автомобилей. Для каждого из них попробуйте добавить явные типы к кодировкам (например, измените `"Horsepower"` на `"Horsepower:Q"`, чтобы график не изменился.

Есть ли графики, которые можно улучшить, изменив тип?

In [ ]:
alt.Chart(cars).mark_bar().encode(
    y='Origin',
    x='mean(Horsepower)'
)

In [ ]:
alt.Chart(cars).mark_line().encode(
    x='Year',
    y='mean(Miles_per_Gallon)',
    color='Origin'
)

In [ ]:
alt.Chart(cars).mark_bar().encode(
    y='Cylinders',
    x='count()',
    color='Origin'
)

In [ ]:
alt.Chart(cars).mark_rect().encode(
    x='Cylinders',
    y='Origin',
    color='count()'
)

*Продолжение следует...*